# **IMPORT**

In [1]:
# Menonaktifkan semua peringatan yang dihasilkan oleh kode Python
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Instalasi library transformers versi 4.18.0
!pip install transformers==4.18.0

# Mengimpor modul yang diperlukan
import json # Untuk bekerja dengan file JSON
import numpy as np # Untuk operasi numerik
import pandas as pd # Untuk manipulasi data
import random # Untuk operasi acak
from matplotlib import pyplot as plt # Untuk visualisasi data
import seaborn as sns # Untuk visualisasi data yang lebih cantik
from wordcloud import WordCloud, STOPWORDS # Untuk membuat word cloud
import missingno as msno # Untuk visualisasi missing values

# Mengimpor modul dari scikit-learn untuk pemrosesan teks dan evaluasi model
from sklearn.feature_extraction.text import CountVectorizer # Untuk konversi teks ke vektor frekuensi kata
from sklearn.model_selection import train_test_split # Untuk membagi data menjadi set pelatihan dan pengujian
from sklearn.metrics import accuracy_score, precision_recall_fscore_support # Untuk evaluasi performa model

# Mengimpor modul dari TensorFlow Keras untuk membangun dan melatih model neural network
from tensorflow.keras.preprocessing import text # Untuk preprocessing teks
from tensorflow.keras.models import Sequential # Untuk membangun model sekuensial
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout # Lapisan-lapisan untuk model
from tensorflow.keras.callbacks import ReduceLROnPlateau # Callback untuk mengurangi learning rate saat terjadi plateau
from tensorflow.keras.preprocessing.sequence import pad_sequences # Mengimpor modul dari TensorFlow untuk preprocessing dan padding sequence

import nltk # Natural Language Toolkit, digunakan untuk NLP
from nltk import word_tokenize # Untuk tokenisasi kata
from nltk.stem import PorterStemmer # Untuk stemming kata

# Mengimpor modul dari PyTorch untuk dataset handling
import torch
from torch.utils.data import Dataset

# Mengimpor modul dari transformers untuk penggunaan model pra-terlatih dari huggingface
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainingArguments
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer
from transformers.trainer_tf import TFTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 43.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


# **DATASET**

In [3]:
# Mount Google Drive untuk mengakses file di Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Fungsi untuk memuat file JSON
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

# Memuat dataset dari Google Drive
filename = '/content/drive/MyDrive/Program/dataset/intents_LAA.json'
intents = load_json_file(filename)

In [5]:
# Fungsi untuk membuat DataFrame dan mengekstrak informasi dari file JSON
def create_and_extract_info(json_file):
    # Membuat DataFrame kosong
    df = pd.DataFrame({
        'Pattern': [],
        'Tag': []
    })

    # Mengisi DataFrame dengan informasi dari JSON
    for intent in json_file['intents']:
        for pattern in intent['patterns']:
            sentence_tag = [pattern, intent['tag']]
            df.loc[len(df.index)] = sentence_tag

    return df

# Menggunakan fungsi untuk membuat dan mengisi DataFrame
df = create_and_extract_info(intents)
df.head() # Menampilkan lima baris pertama dari DataFrame

,Pattern,Tag
0,Halo!,Greetings
1,Hai!,Greetings
2,assamualaikaum,Greetings
3,Hola,Greetings
4,Permisi,Greetings


# **DATA PREPROCESSING**

In [6]:
labels = df['Tag'].unique().tolist() # Mendapatkan daftar unik dari 'Tag' di DataFrame
labels = [s.strip() for s in labels] # Menghapus spasi di awal dan akhir setiap label
num_labels = len(labels) # Menghitung jumlah label
id2label = {id:label for id, label in enumerate(labels)} # Membuat kamus yang memetakan id ke label
label2id = {label:id for id, label in enumerate(labels)} # Membuat kamus yang memetakan label ke id

In [7]:
id2label # Menampilkan kamus id ke label

{0: 'Greetings',
 1: 'name',
 2: 'Yudisium',
 3: 'Persyaratan Yudisium',
 4: 'Predikat Cumlaude',
 5: 'Kehadiran Yudisium',
 6: 'Hasil Sidang Yudisium',
 7: 'Waktu Pendaftaran Yudisium',
 8: 'Yudisium Pending',
 9: 'Pengajuan Similarity',
 10: 'Hasil Cek Similarity',
 11: 'Batas Maksimum Similarity',
 12: 'Similarity Score Lebih dari 20%',
 13: 'Status Similarity Rejected',
 14: 'Kerja Praktek',
 15: 'Surat Pengantar Kerja Praktek',
 16: 'Waktu Pelaksanaan Kerja Praktek',
 17: 'Dosen Pembimbing Kerja Praktek',
 18: 'Pelaksanaan KP Tidak Sesuai Timeline',
 19: 'Syarat SK Bimbingan TA',
 20: 'Mendapatkan SK Bimbingan TA',
 21: 'Masa Berlaku SK Bimbingan Habis',
 22: 'SK Bimbingan Tidak Bisa Diperpanjang',
 23: 'Perubahan Dosen Pembimbing TA',
 24: 'Perubahan Judul TA',
 25: 'Waktu Sidang TA',
 26: 'Pendaftaran Sidang TA',
 27: 'Jadwal Seminar Internal',
 28: 'Sertifikat Seminar Internal',
 29: 'Aktivasi Mahasiswa',
 30: 'Dispensasi',
 31: 'Transkrip Sementara',
 32: 'SKL',
 33: 'Keringan

In [8]:
label2id # Menampilkan kamus label ke id

{'Greetings': 0,
 'name': 1,
 'Yudisium': 2,
 'Persyaratan Yudisium': 3,
 'Predikat Cumlaude': 4,
 'Kehadiran Yudisium': 5,
 'Hasil Sidang Yudisium': 6,
 'Waktu Pendaftaran Yudisium': 7,
 'Yudisium Pending': 8,
 'Pengajuan Similarity': 9,
 'Hasil Cek Similarity': 10,
 'Batas Maksimum Similarity': 11,
 'Similarity Score Lebih dari 20%': 12,
 'Status Similarity Rejected': 13,
 'Kerja Praktek': 14,
 'Surat Pengantar Kerja Praktek': 15,
 'Waktu Pelaksanaan Kerja Praktek': 16,
 'Dosen Pembimbing Kerja Praktek': 17,
 'Pelaksanaan KP Tidak Sesuai Timeline': 18,
 'Syarat SK Bimbingan TA': 19,
 'Mendapatkan SK Bimbingan TA': 20,
 'Masa Berlaku SK Bimbingan Habis': 21,
 'SK Bimbingan Tidak Bisa Diperpanjang': 22,
 'Perubahan Dosen Pembimbing TA': 23,
 'Perubahan Judul TA': 24,
 'Waktu Sidang TA': 25,
 'Pendaftaran Sidang TA': 26,
 'Jadwal Seminar Internal': 27,
 'Sertifikat Seminar Internal': 28,
 'Aktivasi Mahasiswa': 29,
 'Dispensasi': 30,
 'Transkrip Sementara': 31,
 'SKL': 32,
 'Keringanan B

In [9]:
# Menambahkan kolom 'labels' ke DataFrame dengan memetakan 'Tag' ke id menggunakan kamus label2id
df['labels'] = df['Tag'].map(lambda x: label2id[x.strip()])
df.head() # Menampilkan lima baris pertama dari DataFrame

,Pattern,Tag,labels
0,Halo!,Greetings,0
1,Hai!,Greetings,0
2,assamualaikaum,Greetings,0
3,Hola,Greetings,0
4,Permisi,Greetings,0


# **DATA SPLITING**

In [10]:
# Memisahkan kolom 'Pattern' sebagai X dan kolom 'labels' sebagai y
X = list(df['Pattern'])
y = list(df['labels'])

In [11]:
# Membagi data menjadi set pelatihan dan pengujian dengan rasio default 75:25
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 123)

# **LOAD PRETRAINED MODEL**

In [12]:
# Menentukan model pra-terlatih yang akan digunakan dan panjang maksimum token yang akan diproses oleh tokenizer
model_name = "indolem/indobert-base-uncased"
max_len = 256

# Menginisialisasi tokenizer dari model pra-terlatih
tokenizer = BertTokenizer.from_pretrained(model_name, max_length=max_len)
# Menginisialisasi model untuk klasifikasi urutan dari model pra-terlatih
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id = label2id)

Downloading:   0%|          | 0.00/229k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indober

# **TOKENIZATION**

In [13]:
train_encoding = tokenizer(X_train, truncation=True, padding=True) # Mengkodekan data pelatihan dengan tokenizer BERT
test_encoding = tokenizer(X_test, truncation=True, padding=True) # Mengkodekan data pengujian dengan tokenizer BERT

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
full_data = tokenizer(X, truncation=True, padding=True) # Mengkodekan seluruh data dengan tokenizer BERT

# **DATA LOADER**

In [15]:
# Kelas DataLoader yang merupakan subclass dari Dataset PyTorch
class DataLoader(Dataset):
    # Inisialisasi kelas dengan encoding dan label
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # Mengambil item dari dataset pada indeks tertentu
    def __getitem__(self, idx):
        # Membuat dictionary item dengan mengubah encoding menjadi tensor
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Menambahkan label ke dalam item
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # Mengembalikan panjang dari dataset
    def __len__(self):
        return len(self.labels)

In [16]:
train_dataloader = DataLoader(train_encoding, y_train) # Membuat DataLoader untuk data pelatihan
test_dataloader = DataLoader(test_encoding, y_test) # Membuat DataLoader untuk data pengujian

In [17]:
# Membuat DataLoader untuk seluruh data (y_test digunakan hanya sebagai placeholder)
fullDataLoader = DataLoader(full_data, y_test)

# **EVALUATION METRICS**

In [18]:
# Fungsi untuk menghitung metrik evaluasi
def compute_metrics(pred):
    labels = pred.label_ids # Mengambil label dari prediksi
    preds = pred.predictions.argmax(-1)  # Mengambil prediksi dengan nilai tertinggi
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro') # Menghitung precision, recall, dan f1-score
    acc = accuracy_score(labels, preds)  # Menghitung akurasi

    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

# **TRAINING ARGS**

In [19]:
# Mengatur argumen untuk pelatihan model
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Program/e50/output',  # Direktori output untuk menyimpan model dan checkpoint
    num_train_epochs=50,  # Jumlah epoch untuk pelatihan
    per_device_train_batch_size=128,  # Ukuran batch per perangkat untuk pelatihan
    per_device_eval_batch_size=128,  # Ukuran batch per perangkat untuk evaluasi
    warmup_steps=50,
    weight_decay=0.05, # Menambahkan L2 regularization dengan nilai weight decay 0.01 untuk membantu mencegah overfitting.
    logging_strategy='steps',  # Strategi untuk mencatat log ('steps' atau 'epoch')
    logging_steps=50,  # Frekuensi mencatat log dalam langkah
    evaluation_strategy="steps",  # Strategi untuk evaluasi ('no', 'steps', 'epoch')
    eval_steps=50,  # Frekuensi evaluasi dalam langkah
    save_strategy="steps",  # Strategi untuk menyimpan model ('no', 'epoch', 'steps')
    load_best_model_at_end=False # Menonaktifkan load best model at end
)

# **TRAINING**

In [20]:
# Membuat Trainer untuk melatih model
trainer = Trainer(
    model=model,  # Model yang akan dilatih
    args=training_args,  # Argumen pelatihan
    train_dataset=train_dataloader,  # Data pelatihan
    eval_dataset=test_dataloader,  # Data evaluasi
    compute_metrics=compute_metrics  # Fungsi untuk menghitung metrik evaluasi
)

In [21]:
# Melatih model
trainer.train()

***** Running training *****
  Num examples = 945
  Num Epochs = 50
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 400


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,3.861300,3.238542,0.487342,0.423392,0.511690,0.504647
100,1.782600,0.494301,0.952532,0.931770,0.933951,0.940789
150,0.259500,0.121271,0.968354,0.961313,0.966003,0.964377
200,0.064000,0.134143,0.962025,0.951395,0.958524,0.955563
250,0.043000,0.108042,0.971519,0.964169,0.967376,0.966781
300,0.028200,0.103302,0.968354,0.957209,0.960966,0.960371
350,0.023700,0.116039,0.971519,0.963908,0.968750,0.966438
400,0.021000,0.115179,0.968354,0.957209,0.960966,0.960371


***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128
***** Running Evaluation *****
  Num examples = 316
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=400, training_loss=0.7604090169072151, metrics={'train_runtime': 114.3695, 'train_samples_per_second': 413.135, 'train_steps_per_second': 3.497, 'total_flos': 777348653760000.0, 'train_loss': 0.7604090169072151, 'epoch': 50.0})

# **EVALUATE MODEL**

In [22]:
# Evaluasi model pada dataset pelatihan dan pengujian
q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]
# Membuat DataFrame untuk hasil evaluasi dan menampilkannya
pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

***** Running Evaluation *****
  Num examples = 945
  Batch size = 128


***** Running Evaluation *****
  Num examples = 316
  Batch size = 128


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.014260,0.996825,0.997239,0.997596,0.997253
test,0.115179,0.968354,0.957209,0.960966,0.960371


In [23]:
def predict(text):
    # Tokenisasi teks input dengan padding dan truncation, dan konversi ke tensor PyTorch
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    # Dapatkan output dari model dengan memberikan input yang sudah di-tokenisasi
    outputs = model(**inputs)
    # Hitung probabilitas dengan menerapkan fungsi softmax pada output
    probs = outputs[0].softmax(1)
    # Temukan indeks label dengan probabilitas tertinggi
    pred_label_idx = probs.argmax()
    # Konversi indeks label menjadi label sebenarnya menggunakan konfigurasi model
    pred_label = model.config.id2label[pred_label_idx.item()]
    # Kembalikan probabilitas, indeks label prediksi, dan label prediksi
    return probs, pred_label_idx, pred_label

In [24]:
text = "hai"
predict(text)

(tensor([[9.8965e-01, 9.8125e-04, 1.3977e-04, 2.3320e-04, 2.0897e-04, 1.1582e-04,
          1.2340e-04, 2.9300e-04, 1.9353e-04, 8.4236e-05, 1.1959e-04, 1.3573e-04,
          1.1424e-04, 9.1379e-05, 1.5380e-04, 1.1766e-04, 3.1547e-04, 1.4997e-04,
          4.0271e-04, 1.5269e-04, 3.4164e-04, 6.0304e-05, 8.9017e-05, 6.2094e-05,
          7.4848e-05, 1.1132e-04, 6.7885e-05, 1.1170e-04, 9.5154e-05, 9.9156e-05,
          1.3682e-04, 1.9935e-04, 1.1984e-04, 2.4571e-04, 2.1734e-04, 3.6632e-05,
          1.9890e-04, 2.5151e-04, 9.5995e-05, 5.8069e-04, 5.5134e-05, 1.5283e-04,
          1.4660e-04, 5.8383e-05, 7.7673e-05, 3.2094e-04, 5.5315e-05, 1.4354e-04,
          1.5431e-04, 1.3837e-04, 8.7870e-05, 1.6396e-03]], device='cuda:0',
        grad_fn=<SoftmaxBackward0>),
 tensor(0, device='cuda:0'),
 'Greetings')

# **SAVE MODEL**

In [25]:
# Menyimpan model yang telah dilatih ke path yang ditentukan
model_path = "/content/drive/MyDrive/Program/e50/chatbot"
trainer.save_model(model_path)
# Menyimpan tokenizer ke path yang sama dengan model
tokenizer.save_pretrained(model_path)

Saving model checkpoint to /content/drive/MyDrive/Program/e50/chatbot
Configuration saved in /content/drive/MyDrive/Program/e50/chatbot/config.json
Model weights saved in /content/drive/MyDrive/Program/e50/chatbot/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Program/e50/chatbot/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Program/e50/chatbot/special_tokens_map.json


('/content/drive/MyDrive/Program/e50/chatbot/tokenizer_config.json',
 '/content/drive/MyDrive/Program/e50/chatbot/special_tokens_map.json',
 '/content/drive/MyDrive/Program/e50/chatbot/vocab.txt',
 '/content/drive/MyDrive/Program/e50/chatbot/added_tokens.json')

# **LOAD MODEL**

In [26]:
model_path = "/content/drive/MyDrive/Program/e50/chatbot"
# Memuat kembali model dan tokenizer dari path yang telah disimpan
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
# Membuat pipeline untuk analisis sentimen menggunakan model dan tokenizer yang telah dilatih
chatbot= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

loading configuration file /content/drive/MyDrive/Program/e50/chatbot/config.json
Model config BertConfig {
  "_name_or_path": "indolem/indobert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Greetings",
    "1": "name",
    "2": "Yudisium",
    "3": "Persyaratan Yudisium",
    "4": "Predikat Cumlaude",
    "5": "Kehadiran Yudisium",
    "6": "Hasil Sidang Yudisium",
    "7": "Waktu Pendaftaran Yudisium",
    "8": "Yudisium Pending",
    "9": "Pengajuan Similarity",
    "10": "Hasil Cek Similarity",
    "11": "Batas Maksimum Similarity",
    "12": "Similarity Score Lebih dari 20%",
    "13": "Status Similarity Rejected",
    "14": "Kerja Praktek",
    "15": "Surat Pengantar Kerja Praktek",
    "16": "Waktu Pelaksanaan Kerja Praktek",
    "17": "D

# **CHAT WITH BOT**

In [27]:
# Fungsi untuk interaksi dengan chatbot
def chat(chatbot):
    # Menampilkan pesan sambutan dari chatbot
    print("Chatbot: Halo! Saya asisten virtual layanan akademik kampus Anda. Jangan ragu untuk bertanya, saya siap membantu Anda dengan informasi dan layanan akademik yang Anda butuhkan")
    print("Ketik 'quit' untuk mengakhiri pembicaraan\n")

    # Membaca input dari pengguna
    text = input("User: ").strip().lower()

    # Loop untuk terus berinteraksi sampai pengguna mengetik 'quit'
    while(text != 'quit'):
        # Mendapatkan skor prediksi dari chatbot
        score = chatbot(text)[0]['score']
        # Jika skor kurang dari 0.8, chatbot memberikan pesan bahwa tidak memahami input pengguna
        if score < 0.8 :
            print("Chatbot: Maaf saya tidak bisa memahami apa yang anda maksud\n", score, "\n")
            text = input("User: ").strip().lower()  # Membaca input selanjutnya dari pengguna
            continue

        # Mendapatkan label dari prediksi chatbot
        label = label2id[chatbot(text)[0]['label']]
        # Memilih respon acak dari intent yang sesuai dengan label
        response = random.choice(intents['intents'][label]['responses'])
         # Menampilkan respon dari chatbot
        print(f"Chatbot: {response}\n", score, "\n")

         # Membaca input selanjutnya dari pengguna
        text = input("User: ").strip().lower()

In [28]:
# Memulai interaksi dengan chatbot
chat(chatbot)

Chatbot: Halo! Saya asisten virtual layanan akademik kampus Anda. Jangan ragu untuk bertanya, saya siap membantu Anda dengan informasi dan layanan akademik yang Anda butuhkan
Ketik 'quit' untuk mengakhiri pembicaraan

User: hai
Chatbot: Halo, bagaimana keadaan hari ini?
 0.9896472096443176 

User: cara cuti akademik
Chatbot: Berikut adalah tahapan pengajuan cuti akademik oleh mahasiswa:<br><ol><li>Akses <a href="https://igracias.telkomuniversity.ac.id" target="_blank">iGracias</a>.</li><li>Pilih menu Registrasi.</li><li>Pilih menu Pengajuan Cuti Mahasiswa.</li><li>Pilih Semester (semester berjalan).</li><li>Input data dan unggah dokumen kelengkapan cuti.</li><li>Klik submit.</li><li>Setelah submit, pada ajuan cuti mahasiswa di ujung kanan ada icon printer untuk cetak form cuti, kemudian lengkapi form cuti tersebut.</li><li>Setelah lengkap, pada ajuan cuti mahasiswa, di bagian ujung kanan ada icon pinsil untuk unggah form cuti.</li><li>Konfirmasi kepada dosen wali untuk menginformasikan